In [ ]:
# https://github.com/zzagir

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from pymongo import MongoClient
import time

# авторизация через аккаунт 'https://account.mail.ru/login'
# авторизация через айфрейм (картинка в картинке) 
# у селениума есть методы для айфрейма. 
# напрямую работать в айфрейме в рамках текущей страницы не получится.
# метод switch_to у драйвера. 
s = Service('../chromedriver.exe')
options = Options()
options.add_argument('start-maximized')
driver = webdriver.Chrome(service=s, options=options)
driver.implicitly_wait(10)

driver.get('https://account.mail.ru/login')

# авторизовались - ввели логин
input = driver.find_element(By.NAME, "username")
input.send_keys("study.ai_172@mail.ru")
input.send_keys(Keys.ENTER)

# ввели пароль
input = driver.find_element(By.NAME, "password")
input.send_keys("NextPassword172#")
input.send_keys(Keys.ENTER)


mails = []

# Выбираются данные по определённым селекторам
for i in range(1):
    time.sleep(3)
    items = driver.find_elements(By.XPATH, "//a[contains(@class,'llc llc_normal')]")
    actions = ActionChains(driver)

    # перебираются письма
    for item in items:
        link = item.get_attribute('href')
        info = item.find_element(By.CLASS_NAME, 'llc__content')
        author = info.find_element(By.CLASS_NAME, 'll-crpt').get_attribute('title')
        subject = info.find_element(By.CLASS_NAME, 'll-sj__normal').text
        date = item.find_element(By.CLASS_NAME, 'llc__item_date').get_attribute('title')
        mail = {'author': author, 'date': date, 'subject': subject, 'link': link}
        mails.append(mail)

# переход к самому нижнему письму
    actions.move_to_element(items[-1])
    actions.perform()

for mail in mails:
    driver.get(mail['link'])
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[contains(@id, '_BODY')]")))
    mail['text'] = driver.find_element(By.XPATH, "//*[contains(@id, '_BODY')]").text
    driver.back()
    WebDriverWait(driver, 5).until(EC.title_contains('Входящие'))

driver.quit()

print()

db_name = 'mails_db'
client = MongoClient('127.0.0.1', 27017)
db = client[db_name]
db.mails_db.insert_many(mails)

In [ ]:
# https://github.com/belkanov

"""
Вариант I
Написать программу, которая собирает входящие письма из своего или тестового
почтового ящика и сложить данные о письмах в базу данных
(от кого, дата отправки, тема письма, текст письма полный)
Вариант II
2) Написать программу, которая собирает товары «В тренде» с сайта техники
mvideo и складывает данные в БД. Сайт можно выбрать и свой.
Главный критерий выбора: динамически загружаемые товары
"""
import logging
from hashlib import sha3_256
from time import sleep
from typing import Optional

import requests
from pymongo import MongoClient
from pymongo.collection import Collection
from pymongo.errors import DuplicateKeyError
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from constants import *

logging.basicConfig(format='%(asctime)s | %(levelname)-8s | %(name)s | %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger('job_scraper')
logger.setLevel(logging.INFO)


def parse_mails(url):
    service = Service(executable_path='./chromedriver_win32_102.0.5005.61.exe')
    options = Options()
    options.add_argument('start-maximized')

    with webdriver.Chrome(service=service, options=options) as driver:
        driver.implicitly_wait(10)

        mails_data = []
        # на простой странице (https://mail.ru) у меня не находил тэги
        # для ввода логопаса.
        # Искал через xpath, id, имена тэгов. Даже самым широким поиском
        # в списке не было искомых. Не понял в чем печаль.
        # Возможно плохо дружит с iframe - хз..
        driver.get(url)

        # авторизируемся
        username_field = driver.find_element(By.CSS_SELECTOR, 'input[name="username"]')
        username_field.send_keys('study.ai_172@mail.ru')
        username_field.send_keys(Keys.ENTER)
        password_field = driver.find_element(By.CSS_SELECTOR, 'input[name="password"]')
        password_field.send_keys('NextPassword172#')
        password_field.send_keys(Keys.ENTER)

        # Решил немного поиграться с интерфейсом.
        # Возможно так менее эффективно, чем просто собрать ссылки,
        # а потом по ним пройтись, но так явно интересней =)
        #
        # ЭТО ЛОМАЕТ ОТОБРАЖЕНИЕ ПИСЕМ ДЛЯ ОСТАЛЬНЫХ УЧАСТНИКОВ
        # поэтому если проверять, то где-нить на отдельном ящике,
        # либо после тестов - вернуть галку обратно

        # кнопка "настройки" слева внизу
        driver.find_element(By.CSS_SELECTOR, 'div.settings').click()
        # галка "Список писем с колонкой письма"
        if driver.find_elements(By.CSS_SELECTOR, 'div[data-test-id="3pane-enabled"]'):
            pass
        else:
            driver.find_element(By.CSS_SELECTOR, 'div[data-test-id="3pane-disabled"]').click()
        # крестик "Закрыть настройки"
        driver.find_element(By.CSS_SELECTOR, 'button[data-test-id="cross"]').click()

        # Искусственное ограничение, чтобы не собирать все 1к+ писем.
        # По-хорошему, если делать это на постоянке, то нас будут интересовать
        # только непрочитанные, которые тоже можно найти.
        max_mails_cnt = 10
        mails_cnt = 0
        prev_mail_link = ''

        # находим первое письмо и открываем его
        driver.find_element(By.CSS_SELECTOR, 'a.llct').click()
        while True:
            # там же есть еще llct_active, но я решил по llct_open.
            # хз куда актив может указать, а llct_open, по логике должен
            # смотреть на открытое, а оно одно в данной ситуации
            current_mail = driver.find_element(By.CSS_SELECTOR, 'a.llct_open')
            # переменная с сылко на текущее письмо
            current_mail_link = current_mail.get_attribute('href')
            # если при переходе на следующее письмо ссылка не изменилась 
            # значит мы достигли последнего письма.
            if prev_mail_link == current_mail_link or mails_cnt >= max_mails_cnt:
                break

            prev_mail_link = current_mail_link
            mails_cnt += 1

            # тема письма, она тут полная.
            # Обрезается видимо JSом или вообще стилями, я хз..
            # Где-то 1 из 5 - не собиралась тема первого письма.
            # Я не стал ковырять - думаю затупы моего ПК.
            mail_title = current_mail.find_element(By.CSS_SELECTOR, 'div.llct__subject').get_attribute('title')
            # от кого (название+ящик)
            mail_from = current_mail.find_element(By.CSS_SELECTOR, 'span.ll-crpt').get_attribute('title')
            # дата отправки. Надо бы еще переделать "Сегодня/Вчера/ХХ мая/..." на
            # адекватные значения, но я не стал - все таки не боевая задача =)
            mail_send_date = current_mail.find_element(By.CSS_SELECTOR, 'div.llct__date').get_attribute('title')
            # текст письма
            # выше в DOM есть "letter__body", но в нем еще какие-то календари и обертки
            # решил их не брать
            mail_body_text = driver.find_element(By.CSS_SELECTOR, 'div.letter-body__body-content').text

            # переходим к следующему письму
            current_mail.send_keys(Keys.ARROW_DOWN)

            mails_data.append({
                'title': mail_title,
                'from': mail_from,
                'send_date': mail_send_date,
                'body_text': mail_body_text,
            })

            # попробуем сразу двух зайцев
            # 1 - будем косить под пользователя и не жать на кнопку 500раз/сек
            # 2 - дадим тексту письма загрузиться (хоть это и не самый кошерный вариант)
            sleep(2)

    return mails_data


def get_mongo_collection(collection_name: str) -> Optional[Collection]:
    client = MongoClient(host=MONGODB_HOST,
                         port=MONGODB_PORT,
                         username=MONGO_INITDB_ROOT_USERNAME,
                         password=MONGO_INITDB_ROOT_PASSWORD)
    db = client[MONGODB_DB_NAME]

    collection = getattr(db, collection_name, None)
    if collection is None:
        raise ValueError(1, f"Коллекция {collection_name} не найдена")

    return collection


def save_to_mongo(data):
    mails_collection = get_mongo_collection(MONGO_COLLECTION)
    for row in data:
        try:
            # При условии преобразования даты в нормальное значение
            # (о чем я писал выше и не делал) - можно получить норм хэш для одного
            # и того же письма, которое мы соберем сегодня, а потом завтра.
            # Можно конечно и ИД брать, который есть в URL или
            # data-id="0:16539077630958936900:0"
            mail_unique_str = ''.join((row['from'], row['send_date']))
            mail_hash = sha3_256(mail_unique_str.encode('utf-8')).hexdigest()
            mails_collection.insert_one({'_id': mail_hash, **row})
        except DuplicateKeyError as e:
            logger.debug('Для ID=%s уже есть запись. Пропускаем', vacancy_id)  # noqa
            pass


def main():
    url = MAIN_URL
    logger.info('Parse mails...')

    try:
        mails_list = parse_mails(url)
    except Exception as e:
        logger.exception(e)
        raise SystemExit(1)

    logger.info('Сохраняю их в Mongo')
    save_to_mongo(mails_list)


if __name__ == '__main__':
    logger.info('--- START')
    main()
    logger.info('--- END')

In [ ]:
# Androkotey

from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import my_functions as my
from tokens import mail_login, mail_password

PATH_TO_CHROMEDRIVER = 'Y:\\My_Programs\\chromedriver_win32\\chromedriver.exe'
url = "https://account.mail.ru/login/"

login = mail_login
password = mail_password

STOP = 0

options = Options()
options.add_argument('start-maximized')
driver = webdriver.Chrome(service=Service(PATH_TO_CHROMEDRIVER), options=options)
driver.implicitly_wait(10)
wait = WebDriverWait(driver, 10)

driver.get(url)

login_field = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[name='username']")))
login_field.send_keys(login)

enter = wait.until(EC.presence_of_element_located((By.XPATH, '//span[contains(text(), "Ввести пароль")]')))
enter.click()

login_password = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[name='password']")))
login_password.send_keys(password)

enter = wait.until(EC.presence_of_element_located((By.XPATH, '//span[contains(text(), "Войти")]')))
enter.click()


def down(i):
    """
    Прокручивает страницу вниз
    """

    try:
        if i == 1:
            anchor = get_letters()
            anchor[len(anchor) // 2].send_keys(Keys.PAGE_UP)
            return
        anchor = get_letters()
        anchor[len(anchor) // 2].send_keys(Keys.PAGE_UP)
        anchor = get_letters()
        anchor[len(anchor) // 2].send_keys(Keys.PAGE_DOWN)
        anchor = get_letters()
        anchor[len(anchor) // 2].send_keys(Keys.PAGE_DOWN)
    except:
        return


def get_letters():
    """
    Получает список объектов, из которых потом можно достать ссылки на письма
    """
    time.sleep(0.2)
    wait.until(EC.presence_of_element_located((By.XPATH, "//a[contains(@class, 'llc_normal')]")))
    letters = driver.find_elements(By.XPATH, "//a[contains(@class, 'llc_normal')]")
    return letters


def add_links(letters, links_list):
    """
    Добавляет ссылки в список. STOP, чтобы вспомнить, что упёрлось в дно
    """

    global STOP
    cnt = 0
    for letter in letters:
        link = letter.get_attribute('href')
        if link not in links_list:
            links_list.append(link)
            cnt += 1
    print(f'Добавлено {cnt} писем')
    print(f'Всего {len(links_list)} писем')
    if cnt == 0:
        STOP += 1
    else:
        STOP = 0


def get_all_letter_links():
    """
    Возвращает список ссылок на все письма
    """

    time.sleep(5)
    links = []
    i = 0
    while STOP != 3:
        add_links(get_letters(), links)
        down(i)
        i += 1

    return links


def get_data(links):
    """
    Заходит в каждое письмо и собирает информацию (очень медлено)
    """

    letters = []
    for i, link in enumerate(links):
        driver.get(url=link)
        letter_info = {}

        time.sleep(1)
        letter_info['title'] = driver.find_element(By.CSS_SELECTOR, "h2[class='thread-subject']").text
        letter_info['from'] = driver.find_element(By.CSS_SELECTOR, "span[class='letter-contact']").text
        letter_info['date'] = driver.find_element(By.CSS_SELECTOR, "div[class='letter__date']").text
        letter_info['full_text'] = driver.find_element(By.CSS_SELECTOR, "div[class='letter__body']").text

        letters.append(letter_info)
        print(f"Обработано {i+1}/{len(links)}")

    return letters


links = get_all_letter_links()
data = get_data(links[:50])

with my.connect_to_mongodb_collection(database_name='hw5', collection_name='letters', delete=False) as collection:
    my.data_insert(collection, data)

driver.close()

In [ ]:
# Androkotey

from selenium.webdriver import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time

import my_functions as my
import re

# отдельная функция которая открывает кнопку в тренде 
def open_trends(driver, steps_down=3):
    """
    Нажимает на кнопку "В тренде"
    """

    time.sleep(5)
    for _ in range(steps_down):
        driver.find_element(By.CLASS_NAME, "popmechanic-desktop").send_keys(Keys.PAGE_DOWN)
    driver.find_element(By.CLASS_NAME, "popmechanic-desktop").send_keys(Keys.PAGE_UP)
    wait = WebDriverWait(driver, 10)
    try:
        button = wait.until(EC.presence_of_element_located((By.XPATH, '//span[contains(text(), "В тренде")]')))
        button.click()
    except TimeoutException:
        print("Не смог нажать на кнопку")
        exit()
    except ElementClickInterceptedException:
        print("Промазал")
        exit()
    time.sleep(1)


def get_data(driver):
    """
    Получает данные из карусели "В тренде"
    """

    carousel = driver.find_element(By.XPATH, '//*[@class="carusel ng-star-inserted"]')
    fields_to_get = ['name', 'price__main']

    products = []  # будет содержать два списка одинаковой длины
    for field in fields_to_get:
        product_field = carousel.find_elements(By.XPATH, f'.//*[contains(@class, "{field}")]')

        # Добавляет в список либо наименование, либо рубли
        products.append(list(map(
            lambda x: int(value) if (value := re.sub(r' (\d+) ₽', r'\1', x.text)).isdigit()
            else x.text,
            product_field
        )))

    return [{'name': name, 'price': price} for name, price in zip(*products)]


def main():
    my_driver = my.ConnectionToDriver(url='https://www.mvideo.ru/').driver
    open_trends(my_driver)
    data = get_data(my_driver)
    with my.connect_to_mongodb_collection(database_name='hw5', collection_name='goods', delete=False) as collection:
        my.data_insert(collection, data, index='name')
    my_driver.close()


if __name__ == '__main__':
    main()